In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from textblob import TextBlob
import re

In [2]:
def remove_mentions(tweet):
    return re.sub('@[A-Za-z0-9_]+'," ",tweet)

def remove_hashtags(tweet): 
    return re.sub('#[A-Za-z0-9_]+'," ",tweet)

def remove_links(tweet): 
    tweet = re.sub('[()!?]'," ",tweet)           
    tweet = re.sub(r'http\S+'," ",tweet)         
    tweet = re.sub(r'www.\S+'," ",tweet)         
    tweet = re.sub(r'\[.*?\]',' ', tweet)        
    return re.sub(r'[^a-z0-9]'," ",tweet)         

def get_sentiment(tweet):
    blob = TextBlob(tweet)
    return blob.sentiment.polarity

def get_sentiment_name(sentiment):
    if sentiment == 0:
        return "Neutral 😊"
    elif sentiment > 0:
        return "Positive🥳"
    else:
        return "Negative🤬"

def load_data():
    df = pd.read_csv('Valorant.csv')
    df.rename(columns={'Text':'Tweets'}, inplace = True)
    df['Tweets'] = df['Tweets'].str.lower()
    df['Tweets'] = df['Tweets'].apply(remove_hashtags)
    df['Tweets'] = df['Tweets'].apply(remove_mentions)
    df['Tweets'] = df['Tweets'].apply(remove_links)
    df['Sentiment'] = df['Tweets'].apply(get_sentiment)
    df['SentimentName'] = df["Sentiment"].apply(get_sentiment_name)
    return df

In [3]:
df = load_data()
df

,Unnamed: 0,Date,Id,Tweets,Length,Likes,Retweets,Source,Sentiment,SentimentName
0,0,02-03-2020 07:57,1.234390e+18,sommer 2020 2020 2020,53,89679,19958,playVALORANT,0.000000,Neutral 😊
1,1,02-03-2020 08:01,1.234390e+18,summer 2020 t 2020 verano de 2020,84,36545,7276,playVALORANT,0.000000,Neutral 😊
2,2,02-03-2020 20:40,1.234580e+18,thank you to everyone who has expressed intere...,139,31719,1312,playVALORANT,0.000000,Neutral 😊
3,3,02-03-2020 20:43,1.234580e+18,rt lol fps valorant ...,140,0,4055,playVALORANT,0.800000,Positive🥳
4,4,02-03-2020 20:45,1.234580e+18,rt for the past six years riot games has b...,140,0,424,playVALORANT,-0.250000,Negative🤬
...,...,...,...,...,...,...,...,...,...,...
357,357,23-04-2020 17:00,1.253370e+18,omen unknown you thought you were safe,78,13258,1299,playVALORANT,0.200000,Positive🥳
358,358,23-04-2020 17:11,1.253370e+18,the official reveal video of omen is now live ...,117,291,21,ValorantUpdates,0.136364,Positive🥳
359,359,23-04-2020 17:13,1.253370e+18,here s more info on the best ways to use omen ...,83,50,2,ValorantUpdates,0.750000,Positive🥳
360,360,23-04-2020 18:55,1.253400e+18,track is panic by stefandr,30,977,17,playVALORANT,0.000000,Neutral 😊


In [4]:
counter = df.SentimentName.reset_index()
c1,c2 = st.columns([1,3])
# c1.write(counter)
counter.value_counts()
counter['Date'] = df['Date']
# counter.set_index('Date')
counter

,index,SentimentName,Date
0,0,Neutral 😊,02-03-2020 07:57
1,1,Neutral 😊,02-03-2020 08:01
2,2,Neutral 😊,02-03-2020 20:40
3,3,Positive🥳,02-03-2020 20:43
4,4,Negative🤬,02-03-2020 20:45
...,...,...,...
357,357,Positive🥳,23-04-2020 17:00
358,358,Positive🥳,23-04-2020 17:11
359,359,Positive🥳,23-04-2020 17:13
360,360,Neutral 😊,23-04-2020 18:55


In [5]:
# counter.set_index('Date',inplace= True)
# counter

In [6]:
counter[counter['Date']<='2020-02-04']

,index,SentimentName,Date
0,0,Neutral 😊,02-03-2020 07:57
1,1,Neutral 😊,02-03-2020 08:01
2,2,Neutral 😊,02-03-2020 20:40
3,3,Positive🥳,02-03-2020 20:43
4,4,Negative🤬,02-03-2020 20:45
...,...,...,...
327,327,Positive🥳,20-04-2020 22:04
328,328,Positive🥳,20-04-2020 22:06
329,329,Positive🥳,20-04-2020 23:21
330,330,Neutral 😊,20-04-2020 23:30


In [7]:
counter = pd.concat([counter,pd.get_dummies(counter['SentimentName'])], axis=1)

In [8]:
counter['Date'] = pd.to_datetime(counter['Date'])
counterdf = counter.set_index('Date').resample('D').sum()
counterdf.replace(0,np.nan,inplace=True)

In [9]:
counterdf.dropna(how='all',inplace=True)

In [10]:
px.funnel(counterdf, counterdf.index, ['Negative🤬','Neutral 😊','Positive🥳'])